In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

import numpy as np
import xgboost as xgb
import seaborn as sns


import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression,LinearRegression 
from xgboost import XGBClassifier,XGBRegressor
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor
import datetime
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

In [ ]:

train = pd.read_csv(r'/kaggle/input/bike-share/TRAIN.csv')
test = pd.read_csv(r'/kaggle/input/bike-share/TEST.csv')

test1 = test.copy()

In [ ]:
df = pd.concat([train,test])

In [ ]:
df['Day'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.day            
df['Month'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.month 
df['Year'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.year
df['dow'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.dayofweek
df['woy'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.weekofyear

In [ ]:
df = df.drop('Index', axis = 1)

In [ ]:
cols= ['Seasons', 'Holiday','Functioning Day']
df[cols]= df[cols].apply(LabelEncoder().fit_transform)

In [ ]:
"""sc = StandardScaler()
df['Temperature(°C)'] = sc.fit_transform(df['Temperature(°C)'].values.reshape(-1, 1))
df['Humidity(%)'] = sc.fit_transform(df['Humidity(%)'].values.reshape(-1, 1))
df['Wind speed (m/s)'] = sc.fit_transform(df['Wind speed (m/s)'].values.reshape(-1, 1))
df['Dew point temperature(°C)'] = sc.fit_transform(df['Dew point temperature(°C)'].values.reshape(-1, 1))"""

skew

In [ ]:
columns =['Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)',
       
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Rented Bike Count', 'Day', 'Month']
for i in columns:
    print(i, df[i].skew())

In [ ]:
columns =['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)',
       'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Rented Bike Count', 'Day', 'Month']
for i in columns:
    print(i, np.log(df[i]).skew())

In [ ]:
columns =['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)',
       'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Rented Bike Count', 'Day', 'Month']
for i in columns:
    print(i, np.sqrt(df[i]).skew())  #wind,solar,snow, renta bike

In [ ]:
df['Rainfall(mm)'] = np.log1p(df['Rainfall(mm)'])
df['Wind speed (m/s)'] = np.log1p(df['Wind speed (m/s)'])         
columns2 =['Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)']#, 'Rented Bike Count']
for i in columns2:
    df[i]= np.sqrt(df[i])

In [ ]:
columns =['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)',
       'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Rented Bike Count', 'Day', 'Month']
for i in columns:
    print(i, df[i].skew())

outliers

In [ ]:
"""out = ['Solar Radiation (MJ/m2)','Wind speed (m/s)']
filtered_entries = np.array([True] * len(train))
for col in out:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    low_limit = Q1 - (IQR * 1.5)
    high_limit = Q3 + (IQR * 1.5)

    filtered_entries = (train[col] >= low_limit) & ((train[col] <= high_limit)) & filtered_entries
    
train = train[filtered_entries]"""

In [ ]:
sns.heatmap(df.corr()) 

new columns

In [ ]:
#night
df['Night'] = 0
df.loc[(df['Hour'] <= 6) | (df['Hour'] > 22), 'Night'] = 1


In [ ]:
#rain
df['Rain'] = 1
df.loc[df['Rainfall(mm)'] == 0,'Rain'] = 0

In [ ]:
#weekday
df['Weekday'] =  1
df['Weekday'] = ((pd.DatetimeIndex(df['Date']).dayofweek) // 5 == 0).astype(float)

In [ ]:
#officetime on weekday
df['Office']=0
df.loc[((df['Hour'] >=9) &  (df['Hour'] <= 17)) & (df['Weekday'] == 1), 'Office'] = 1

In [ ]:
#Rush morning on weekday
df['Rush_m']=0
df.loc[((df['Hour']>=6)&(df['Hour']<10))& (df['Weekday'] == 1), 'Rush_m'] = 1

In [ ]:
#rush evning on weekday  15 19
df['Rush_e']=0
df.loc[((df['Hour']>=15)&(df['Hour']<19))& (df['Weekday'] == 1), 'Rush_e'] = 1

In [ ]:
#season high !!!
df["SeasonH"] = 0
df.loc[df["Seasons"] == 0,"SeasonH"] =1

In [ ]:
#month high !!!
df["MonthH"] = 0
df.loc[(df["Month"] > 4)& (df['Month']<10),"MonthH"] =1

In [ ]:
#rush weekeend!!!
df['Rush_w']= 0
df.loc[((df['Hour']>=17)&(df['Hour']<21))&(df['Weekday'] == 0),'Rush_w']= 1

In [ ]:
df['Hour'+ '_sin'] = np.sin(2 * np.pi * df['Hour'] /23)
df['Hour' + '_cos'] = np.cos(2 * np.pi * df['Hour'] /23)

In [ ]:
df['Month' + '_sin'] = np.sin(2 * np.pi * df['Month'] /12)
df['Month'  + '_cos'] = np.cos(2 * np.pi * df['Month' ] / 12)

In [ ]:
df['count_log'] = np.log1p(df['Rented Bike Count'])

In [ ]:
df

In [ ]:
train = df.iloc[:7032, :]
test = df.iloc[7032:,:]
test = test.drop(['Rented Bike Count','count_log'], axis =1)

In [ ]:
X = train.drop(['Rented Bike Count','count_log','Date'], axis =1)
y =train['count_log']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.info()

In [ ]:
params = {'colsample_bytree': [0.8],
  'learning_rate': [0.1],
  'max_depth':[ 20],
  'min_child_weight':[ 9],
  'n_estimators': [1100],
  'seed': [32],
  'subsample':[ 0.7]} 
model =  xgb.XGBRegressor(params =params, objective='reg:squarederror',nthread=4,
                       verbose=1, n_jobs = -1,
                        cv = 5,
                        error_score='rmse')


model.fit(X_train,y_train)
pred_y = model.predict(X_test)


model.score(X_train, y_train)
rmse = np.sqrt(mean_squared_error(y_test, pred_y)) 
print("RMSE : % f" %(rmse)) 

eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["rmse","error"]
result= model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)
result

In [ ]:
from xgboost import plot_importance
plot_importance(model,importance_type='gain')